#Transfer learning VGG 16 and VGG 19 using Keras

In [ ]:
'''
from tensroflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth= True
session = InteractiveSession(config = config)
'''
# local machine ma gpu cha bhane tesko lagi yo code lekhne

'\nfrom tensroflow.compat.v1 import ConfigProto\nfrom tensorflow.compat.v1 import InteractiveSession\n\nconfig = ConfigProto()\nconfig.gpu_options.per_process_gpu_memory_fraction = 0.5\nconfig.gpu_options.allow_growth= True\nsession = InteractiveSession(config = config)\n'

In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/cotton disease prediction-deep learning dataset/train'
valid_path = '/content/drive/MyDrive/cotton disease prediction-deep learning dataset/test'

In [ ]:
# Import the resnet 50 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights
for layer in resnet.layers:
  layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/cotton disease prediction-deep learning dataset/train/*')

In [ ]:
# our layers - you can add more of you want
x = Flatten()(resnet.output)

In [ ]:
len(folders)

4

In [ ]:
prediction = Dense(len(folders), activation = 'softmax')(x)

# create a model object
model = Model(inputs = resnet.input, outputs = prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
# use the image data generator to import the images from the dataset

from keras.preprocessing.image import ImageDataGenerator, load_img

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range=0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/cotton disease prediction-deep learning dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1956 images belonging to 4 classes.


In [ ]:

test_set = train_datagen.flow_from_directory('/content/drive/MyDrive/cotton disease prediction-deep learning dataset/test',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode='categorical')

Found 18 images belonging to 4 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute

nb_train_samples = 960
nb_validation_samples = 32
epochs = 10
batch_size = 32

history = model.fit_generator(
    training_set,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = test_set,
    validation_steps = nb_validation_samples // batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
30/30 [==============================] - 597s 20s/step - loss: 7.5946 - accuracy: 0.2483 - val_loss: 2.9425 - val_accuracy: 0.3889
Epoch 2/10
30/30 [==============================] - 311s 10s/step - loss: 1.7492 - accuracy: 0.4328 - val_loss: 1.2785 - val_accuracy: 0.6667
Epoch 3/10
30/30 [==============================] - 184s 6s/step - loss: 1.1714 - accuracy: 0.5868 - val_loss: 0.9824 - val_accuracy: 0.6111
Epoch 4/10
30/30 [==============================] - 150s 5s/step - loss: 0.9769 - accuracy: 0.6226 - val_loss: 1.1558 - val_accuracy: 0.5556
Epoch 5/10
30/30 [==============================] - 146s 5s/step - loss: 0.8548 - accuracy: 0.6362 - val_loss: 1.1570 - val_accuracy: 0.5000
Epoch 6/10
30/30 [==============================] - 148s 5s/step - loss: 0.8955 - accuracy: 0.6241 - val_loss: 1.3398 - val_accuracy: 0.5000
Epoch 7/10
30/30 [==============================] - 148s 5s/step - loss: 0.9229 - accuracy: 0.6094 - val_loss: 0.9335 - val_accuracy: 0.6111
Epoch 8/10
